In [1]:
import pyspark
import pyspark.sql.types as T
from pyspark.sql import SparkSession, Row, functions as F
import collections
import functools
import operator
import json
from kpi.zte import *


In [2]:
spark = SparkSession.builder\
    .config("spark.jars", "/Users/khqp/Documents/GitHub/spark/drivers/postgresql-42.5.4.jar")\
    .config("spark.executor.extraClassPath", "/Users/khqp/Documents/GitHub/spark/drivers/postgresql-42.5.4.jar") \
    .master('local') \
    .appName("PySpark_Postgres_test").getOrCreate()


    # .schema(T.StructType([
    #     T.StructField('id',T.StringType()),
    #     T.StructField('enodeb_id', T.StringType()),
    #     T.StructField('cell_id',T.StringType()),
    #     T.StructField('duration', T.IntegerType()),
    #     T.StructField('counters', T.D()),
    #     ]))\
    


23/03/06 14:57:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/pasti") \
    .option("dbtable", "test_zte") \
    .option("user", "postgres") \
    .option("password", "root") \
    .option("driver", "org.postgresql.Driver") \
    .load()


In [7]:
@F.udf()
def sum_json(x) : 
    result = dict(functools.reduce(operator.add,
         map(collections.Counter, map(lambda a : json.loads(a), x))))
    return json.dumps(result)


a = df \
    .groupBy('enodeb_id') \
    .agg(sum_json(F.collect_list('counters')).alias('counters'), F.sum('duration').alias('duration')) \
    .withColumns({
        'C373230700': F.when(F.get_json_object('counters', '$.C373230700').isNotNull(), F.get_json_object('counters', '$.C373230700')).otherwise(0),
        'kpi': F.to_json(
            F.struct((100 * (F.when(F.get_json_object('counters', '$.C373230700').isNotNull(),
                                    F.get_json_object('counters', '$.C373230700')).otherwise(0) / F.col('duration'))).alias('avail')),
        )
        } ) \
    .show(100)
    # .sort(df.started_at.asc()) \


+---------+--------------------+--------+----------+---+
|enodeb_id|            counters|duration|C373230700|kpi|
+---------+--------------------+--------+----------+---+
|  1039260|{"C373200000": 55...|    3600|      3600| {}|
|  1046001|{"C373200000": 4,...|     900|       900| {}|
|  1046002|{"C373200000": 69...|    1800|      1800| {}|
|  1046003|{"C373200000": 57...|    1800|      1800| {}|
|  1046004|{"C373200000": 13...|    1800|      1800| {}|
|  1046005|{"C373200000": 18...|    1800|      1800| {}|
|  1046006|{"C373200000": 19...|    3600|      3600| {}|
|  1046007|{"C373200000": 19...|    1800|      1800| {}|
|  1046008|{"C373200000": 18...|    3600|      3600| {}|
|  1046009|{"C373200000": 9,...|    1800|      1800| {}|
|  1046010|{"C373200000": 5,...|    1800|      1800| {}|
|  1046011|{"C373200000": 7,...|    1800|      1800| {}|
|  1046012|{"C373200000": 12...|    3600|      3600| {}|
|  1046013|{"C373200000": 10...|    3600|      3600| {}|
|  1046014|{"C373200000": 25...